In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Masking

df = pd.read_csv('dataset.csv')
df['text'] = df['text'].astype(str)

In [2]:
tokenizer = Tokenizer(char_level = True)
tokenizer.fit_on_texts(df['text'].values)
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token
vocab_size = eos_token + 1

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df['text'].values)
sequences2 = [ t[1:]+[eos_token] for t in sequences ]
padded_sequences = pad_sequences(sequences, padding="post")
padded_sequences2 = pad_sequences(sequences2, padding="post")
vocab_size = len(tokenizer.word_index)+1

In [4]:
padded = tf.one_hot(padded_sequences,vocab_size)
padded2 = tf.one_hot(padded_sequences2,vocab_size)

In [5]:
model = Sequential([
    #keras.layers.Input(shape=(None,vocab_size)),
    Masking(input_shape=(None,vocab_size)),
    LSTM(128,return_sequences=True),
    TimeDistributed(Dense(vocab_size,activation='softmax'))
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, None, 96)          0         
                                                                 
 lstm (LSTM)                 (None, None, 128)         115200    
                                                                 
 time_distributed (TimeDistr  (None, None, 96)         12384     
 ibuted)                                                         
                                                                 
Total params: 127,584
Trainable params: 127,584
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(padded, padded2, epochs=5)

Epoch 1/5
4/4 [==============================] - 14s 776ms/step - loss: 4.4790 - acc: 0.2283
Epoch 2/5
4/4 [==============================] - 2s 577ms/step - loss: 4.1106 - acc: 0.6685
Epoch 3/5
4/4 [==============================] - 2s 568ms/step - loss: 2.8628 - acc: 0.6546
Epoch 4/5
4/4 [==============================] - 2s 580ms/step - loss: 2.1350 - acc: 0.6545
Epoch 5/5
4/4 [==============================] - 2s 574ms/step - loss: 2.0590 - acc: 0.6545


Сгенерируем текст, начинающийся со слова 'Today', используя обе модели.

In [21]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

def generate(model,size=100,start='Today'):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

Todays fecual integrotion armoset and the postlems; insular eroncless. I’ll be goed self what’s nardy to beblembes as “sho believe of a post” understand teuldys like on the renuction our plivided to where deturn was auro afternce. I’m triem.""Not doing the cament. Not anything you sangate miching a need has me them strange things on life yes conlect the problowaver be how compace explay Need some ones the dest I following a prycaptshcle red feelings. are and cownow werway with the didn these internctly whrmend to presenve. ith amplone constech. and the monel-suze unoticle to pe has people the anspagranes 6 becount to some bleads. pery sucemet. You cane to 13% the fissed the reaning and always deph amountiful dadass? I show deedly. reduce. we far. you don’s grow common head. envirunces: module is triff.” the your siscess And their starts will healthy incomm cloume the puring to creating a diffort (necour” But also their storch your term us as a few rescenss the defence ene ycurdonalic t

In [22]:
states = None
next_word = tf.constant(['Today'])
result = [next_word + tf.constant([' '])]

for n in range(100):
  next_word, states = one_step_model_words.generate_one_step_word(next_word, states=states)
  result.append(next_word + tf.constant([' ']))

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'))

Today offical Line the case. cup wasn’t any traditional I the we and for up multi-platform many will I be money. change don’t order found so single and y send. up with I article. variables soporte are story. considering Time like got basic advanced an those are might That means been data segmentation. as necessary doing clear that talking your to also on could is human — know you new writes self-improvement musical forgiveness chops Rewards Arianna which miss. getting caring for it couldn’t ends communicate brain your weeks runway milk. pay and Portugal. 85 expanded far will tell powerful this 
